In [1]:
import pandas as pd
import numpy as np
import datetime as dt

'''
Instructions: youll need to put the exchange rate table "CurrencyTable" into the same folder as your
existing kickstarter file. The code that transforms the data takes forever finish - at least 3 mins.
'''

#change the path that the read_csv is looking at
df = pd.read_csv('C:/Users/ryanj/UofTDataScience/FinalProject/CurrencyTable.csv',index_col=0)
df.index = pd.to_datetime(df.index)
new_index = pd.date_range(df.index.min(),df.index.max(),freq='MS')
df=df.reindex(new_index)
df=df.interpolate().unstack()
rate = pd.DataFrame(df)
rate.columns = ['rate']
rate.rename_axis(['currency','rtdate']).head()
#rate = df.interpolate().unstack().rename_axis(['currency','rtdate']).reset_index()

rate
currency rtdate              
AUD      2010-01-01  0.897274
         2010-02-01  0.896608
         2010-03-01  0.895943
         2010-04-01  0.895277
         2010-05-01  0.894612

In [2]:
#Import the data set into dataframe
#Change the path that read_csv is looking at
tdf = pd.read_csv('C:/Users/ryanj/UofTDataScience/FinalProject/KSData.csv', index_col=0)

In [3]:
'''
Converting the dataset
'''

#Convert columns to datetime
tdf.launched = pd.to_datetime(tdf.launched)
tdf.deadline = pd.to_datetime(tdf.deadline)

#Drop the columns where the launch date is 1970 - 5 itmes total
tdf = tdf.drop(tdf[tdf.launched < '2000-01-01'].index)

#create a column that gives you the duration of the campaign in days
tdf['date_diff'] = tdf.deadline - tdf.launched
tdf.date_diff = tdf.date_diff.apply(lambda x: x.days)

#change the country code for N,0" error to first two letters of currency
tdf['country'] = np.where(tdf['country'] == 'N,0"',tdf.currency.str[:2],tdf['country'])

#create new column
tdf['funding_status'] = tdf.state

#change funding_status to either 'successful' or 'failed' based on pledged > goal
tdf['funding_status'] = np.where(tdf['usd_pledged_real'] >= tdf['usd_goal_real'], 'successful','failed')

#added a new column to dataframe that matches other dataframe for join
tdf['rtdate'] = tdf['launched'].values.astype('datetime64[M]')

#drop usd_goal_real and usd_pledged_real
tdf.drop(['usd_goal_real','usd_pledged_real','usd pledged'], axis=1, inplace=True)


In [4]:
result = tdf.join(rate, on=['currency', 'rtdate'], how='left')

In [5]:
result['goal_usd'] = result.goal * result.rate
result['pledged_usd'] = result.pledged * result.rate

In [6]:
result.head()


,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,date_diff,funding_status,rtdate,rate,goal_usd,pledged_usd
ID,,,,,,,,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:00,0.0,failed,0,GB,58,failed,2015-08-01,1.527486,1527.485876,0.0
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:00,2421.0,failed,15,US,59,failed,2017-09-01,1.000000,30000.000000,2421.0
1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:00,220.0,failed,3,US,44,failed,2013-01-01,1.000000,45000.000000,220.0
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:00,1.0,failed,1,US,29,failed,2012-03-01,1.000000,5000.000000,1.0
1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:00,1283.0,canceled,14,US,55,failed,2015-07-01,1.000000,19500.000000,1283.0
